### Iniciando SparkSession

In [0]:
from pyspark.sql import SparkSession

# Cria uma SparkSession com as configurações necessárias para o Delta Lake
spark = SparkSession.builder \
    .appName("Leitura Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

#### Número de Núcleos

In [0]:
num_cores = sc._jsc.sc().getExecutorMemoryStatus().keySet().size()
print(f"Número de núcleos no cluster: {num_cores}")


Número de núcleos no cluster: 1


####Evidência de Fato Vendas

In [0]:
from delta.tables import DeltaTable
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"
delta_table = DeltaTable.forPath(spark, f"{delta_path}/fato_vendas")
delta_table.toDF().show()

+----------+----------+------------+-----------+-------------+-----------+--------+-------------+-------------+-----------+----+---+----------------+
| DataVenda|sk_produto|sk_categoria|sk_segmento|sk_fabricante| sk_cliente|Unidades|PrecoUnitario|CustoUnitario|TotalVendas| Ano|Mes|data_atualizacao|
+----------+----------+------------+-----------+-------------+-----------+--------+-------------+-------------+-----------+----+---+----------------+
|2011-04-30|        79|           2|          6|            1| 8589940977|       1|       124.42|        90.83|     124.42|2011|  4|            null|
|2011-04-20|        79|           2|          6|            1|       7740|       1|       124.42|        90.83|     124.42|2011|  4|            null|
|2011-04-17|        79|           2|          6|            1|51539614147|       1|       124.42|        90.83|     124.42|2011|  4|            null|
|2011-04-03|        79|           2|          6|            1|42949677507|       1|       124.42|   

####Evidência de Dim Produto

In [0]:
from delta.tables import DeltaTable
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"
dim_produto_df = DeltaTable.forPath(spark, f"{delta_path}/dim_produto")
dim_produto_df.toDF().show()

+---------+-------+---------+----------+----------------+
|IDProduto|Produto|Categoria|sk_produto|data_atualizacao|
+---------+-------+---------+----------+----------------+
+---------+-------+---------+----------+----------------+



####Evidência de Dim Geografia

In [0]:
from delta.tables import DeltaTable
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"
dim_geografia_df = DeltaTable.forPath(spark, f"{delta_path}/dim_geografia")
dim_geografia_df.toDF().show()

+------+------+------+--------+----+------------+------------+----------------+
|Cidade|Estado|Regiao|Distrito|Pais|CodigoPostal|sk_geografia|data_atualizacao|
+------+------+------+--------+----+------------+------------+----------------+
+------+------+------+--------+----+------------+------------+----------------+



####Evidência de Dim Categoria

In [0]:
from delta.tables import DeltaTable
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"
dim_categoria_df = DeltaTable.forPath(spark, f"{delta_path}/dim_categoria")
dim_categoria_df.toDF().show()


+---------+------------+----------------+
|Categoria|sk_categoria|data_atualizacao|
+---------+------------+----------------+
+---------+------------+----------------+



####Evidência de Dim Cliente

In [0]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"
#dim_cliente_df = DeltaTable.forPath(spark, f"{delta_path}/dim_cliente")
dim_cliente_df = spark.read.format("delta").load(delta_path+"/dim_cliente")
# Conte o número de linhas
#dim_cliente_df.count()
display(dim_cliente_df)


IDCliente,Nome,Email,sk_geografia,sk_cliente,data_atualizacao


####Evidência de Dim Fabricante

In [0]:
from delta.tables import DeltaTable
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"
dim_fabricante_df = DeltaTable.forPath(spark, f"{delta_path}/dim_fabricante")
dim_fabricante_df.toDF().show()

+------------+----------+-------------+----------------+
|IDFabricante|Fabricante|sk_fabricante|data_atualizacao|
+------------+----------+-------------+----------------+
+------------+----------+-------------+----------------+



####Evidência de Dim Segmento

In [0]:
from delta.tables import DeltaTable
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"
dim_segmento_df = DeltaTable.forPath(spark, f"{delta_path}/dim_segmento")
dim_segmento_df.toDF().show()

+--------+-----------+----------------+
|Segmento|sk_segmento|data_atualizacao|
+--------+-----------+----------------+
+--------+-----------+----------------+



%md
**Pontos Importantes para Otimizar a Performance**
 
 **Particionamento**: Definir partições adequadas para evitar leituras desnecessárias e melhorar a performance de consultas.

 **Codec de compressão**: Usar Snappy, pois oferece boa performance de compressão e descompressão.

 **Shuffle partitions**: Definido um valor fixo para spark.sql.shuffle.partitions para melhorar o paralelismo durante operações como joins e agregações.
 Além disso, podemos explorar técnicas como cache para tabelas pequenas (dimensões) que são frequentemente acessadas, e broadcast join para otimizar joins entre a tabela fato e as tabelas de dimensões

### Otimização de Leitura com predicate pushdown:
- É importante certificar que as consultas estão aproveitando o predicate pushdown, o que significa que os filtros são aplicados diretamente ao ler os dados, melhorando a eficiência.


In [0]:
# Utilizando predicate pushdown para otimizar a consulta
# Caminho para o diretório dos arquivos Delta
gold_path = "dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas"
df_filtrado = spark.read.format("delta").load(gold_path).filter("Ano = 2012 AND Mes = 10")

display(df_filtrado)

DataVenda,sk_produto,sk_categoria,sk_segmento,sk_fabricante,sk_cliente,Unidades,PrecoUnitario,CustoUnitario,TotalVendas,Ano,Mes,data_atualizacao
2012-10-01,84,2,6,1,17179890459,1,102.37,74.73,102.37,2012,10,null
2012-10-08,84,2,6,1,42949695217,1,102.37,74.73,102.37,2012,10,null
2012-10-31,84,2,6,1,42949695477,1,102.37,74.73,102.37,2012,10,null
2012-10-10,84,2,6,1,20709,1,102.37,74.73,102.37,2012,10,null
2012-10-28,84,2,6,1,21962,1,102.37,74.73,102.37,2012,10,null
2012-10-19,84,2,6,1,60129566040,1,102.37,74.73,102.37,2012,10,null
2012-10-14,84,2,6,1,17179890307,1,102.37,74.73,102.37,2012,10,null
2012-10-22,84,2,6,1,60129564373,1,102.37,74.73,102.37,2012,10,null
2012-10-23,84,2,6,1,51539629022,1,102.37,74.73,102.37,2012,10,null
2012-10-19,84,2,6,1,8589955108,1,102.37,74.73,102.37,2012,10,null


#### Broadcast join
**Explicação:**
**1. Broadcast Join:**

- O broadcast() é aplicado às tabelas de <b>dimensões</b> (dim_produto_df e dim_cliente_df). Isso replica as tabelas de dimensão para todos os nós, permitindo que as junções sejam realizadas localmente em cada nó, sem necessidade de comunicação entre nós, o que melhora a performance em clusters distribuídos.

**2. Junção com Broadcast:**

- As junções são feitas entre as colunas de chave original (IDProduto, IDCliente) e as tabelas de dimensão para obter as chaves substitutas (SK_Produto, SK_Cliente).

**3. Particionamento:**

- Foi criado colunas de Ano e Mês para otimizar o armazenamento da tabela fato e melhorar o desempenho em consultas temporais. A tabela é particionada por essas colunas.

**Vantagens do Broadcast Join:**

- Reduz a movimentação de dados durante a operação de junção, pois as dimensões pequenas são replicadas para todos os nós.
- Aumenta a performance quando as tabelas de dimensão são significativamente menores que a tabela fato, o que é o caso comum em arquiteturas de data warehouse.

**Desvantagens do Broadcast Join:**
- Limitação de Memória: O DataFrame menor deve caber na memória de todos os nós. Se o DataFrame for muito grande, pode causar erros de falta de memória

In [0]:
from pyspark.sql.functions import year, sum, broadcast,desc
from pyspark.sql import SparkSession

# Leitura das tabelas Delta
vendas_df = spark.read.format("delta").load("/mnt/lhdw/gold/vendas_delta/fato_vendas")
categoria_df = spark.read.format("delta").load("/mnt/lhdw/gold/vendas_delta/dim_categoria")

# Usar broadcast para a tabela categoria
 
categoria_df = broadcast(categoria_df)

# Realizar o join entre as tabelas
joined_df = vendas_df.join(categoria_df, vendas_df.sk_categoria == categoria_df.sk_categoria)

# Agrupar por categoria e ano e calcular a soma do total de vendas
resultado_df = joined_df.groupBy("Categoria", "Ano")\
        .agg(sum("TotalVendas").alias("TotalVendas"))\
        .orderBy("Ano",desc("TotalVendas"))


display(resultado_df)

Categoria,Ano,TotalVendas


**Melhorias de Performance**

Filtros de Partição: Se houver partições específicas que deseja ler, aplicar filtros nas partições pode reduzir significativamente o tempo de leitura.
Reparticionamento: Se os dados estiverem distribuídos de forma desigual, pode usar repartition() para redistribuir o DataFrame com base em uma coluna-chave.

###Performance com PySpark

### 1. Use DataFrame/Dataset em vez de RDD
Os DataFrames e Datasets são mais eficientes que os RDDs, pois incluem otimizações automáticas e um motor de execução otimizado. Eles permitem um melhor gerenciamento de memória e execução mais rápida.

### 2. Evite UDFs (User Defined Functions)
As UDFs podem ser lentas porque não são otimizadas pelo Catalyst Optimizer do Spark. Sempre que possível, use as funções internas do Spark SQL, que são mais eficientes.

### 3. Use `coalesce()` em vez de `repartition()`
O `coalesce()` é mais eficiente que o `repartition()` para reduzir o número de partições, pois evita o shuffle de dados.

### 4. Cache de Dados
Cache os DataFrames que são reutilizados várias vezes em suas operações. Isso evita a re-leitura dos dados do disco e melhora o desempenho.

### 5. Reduza Operações de Shuffle
Operações de shuffle, como `groupByKey` e `reduceByKey`, podem ser caras. Use `mapPartitions` e `reduceByKey` sempre que possível para minimizar o shuffle.

### 6. Ajuste o Número de Partições
Ajuste o número de partições para equilibrar a carga de trabalho entre os executores. Um número inadequado de partições pode levar a um uso ineficiente dos recursos.

### 7. Use Formatos de Dados Serializados
Formatos de dados como Parquet e ORC são mais eficientes para leitura e escrita, pois são compactados e otimizados para consultas.

### 8. Ajuste as Configurações do Spark
Ajuste configurações como `spark.executor.memory`, `spark.executor.cores` e `spark.sql.shuffle.partitions` para otimizar o uso de recursos.

### 9. Utilize a Adaptive Query Execution (AQE)
A AQE permite que o Spark ajuste dinamicamente o plano de execução das consultas com base nas estatísticas de tempo de execução, melhorando o desempenho.

Implementar essas práticas pode ajudar a melhorar significativamente o desempenho de suas aplicações PySpark.